$$\textrm{Joaquin Peñuela Parra}$$
$$\textrm{University of Los Andes}$$
$$\textrm{High Energy Physics Group: Phenomenology of Particles}$$

This code was written to be running in Docker. If you do not have a Docker inside hep-server2 please refer to: https://github.com/Phenomenology-group-uniandes/Tutoriales_Generales

$\textbf{Preliminaries}$ 

The libraries used here are:

In [ ]:
import numpy as np
import shutil
import pandas as pd
import glob
import os

personal_folder="/disco4/personal_folders/Joaquin"

In [ ]:
M = ["500", "1500", "2500"]
g_U = ["0.25", "0.5", "1.0", "2.0", "3.0"]

In [ ]:
ruta_modelo = "/disco1/SIMULACIONES/LeptoQuarks/UFOs/Mod2_VLQ_UFO"
ruta_pythia = "/disco1/SIMULACIONES/LeptoQuarks/pythia8_card.dat"
ruta_delphes = "/disco1/SIMULACIONES/LeptoQuarks/delphes_card_CMS-0.7bTag.dat"

### p p > lq ta @0

In [ ]:
linea_modelo = "import model " + ruta_modelo
linea_def1 = "define lq = vlq vlq~"
linea_def2 = "define ta = ta+ ta-"
linea_gen = "generate p p > lq ta @0"
lineas_definiciones = [linea_modelo, linea_def1, linea_def2, linea_gen]

lineas_outputs = []
for i in range(len(M)):
    for j in range(len(g_U)):
        name="M" + M[i].replace('.','_') + "_gU" + g_U[j].replace('.','_')
        path=os.path.join(personal_folder,"Data_300K_13_TeV","lq_ta",name)
        output_ij = f"output {path} --no-jpeg"
        lineas_outputs.append(output_ij)
lineas = np.concatenate((lineas_definiciones, lineas_outputs))

script = open ('folders_data_lq_ta_M_gU.txt','w')
script.write(lineas[0])

for i in range(1,len(lineas)):
    script.write('\n' + lineas[i])
script.close()

In [ ]:
!./../../../Collider/MG5_aMC_v3_1_0/bin/mg5_aMC folders_data_lq_ta_M_gU.txt

In [ ]:
!rm folders_data_lq_ta_M_gU.txt

In [ ]:
!mkdir "/disco4/personal_folders/Joaquin/Data_300K_13_TeV/lq_ta/Cross_Sections/"

In [ ]:
seeds = []

def semilla():
    seed = np.random.randint(1,5000)
    if (seed in seeds):
        return semilla()
    return seed

for k in range(0,6):            
    for i in range(len(M)):
        for j in range(len(g_U)):
            
            etiqueta = "M" + M[i].replace('.','_') + "_gU" + g_U[j].replace('.','_')
            nombre_script = 'generate_data_lq_ta_' + etiqueta + '.txt'
            script = open (nombre_script,'w')

            launch = "launch /disco4/personal_folders/Joaquin/Data_300K_13_TeV/lq_ta/" + etiqueta + " -m"
            nodos = "10"
            shower = "shower = Pythia8"
            detector = "detector = Delphes"
            done = "done"

            seed = semilla()
            seeds.append(seed)
            param1 = "set iseed " + str(seed)
            param2 = "set nevents 50000"
            param3 = "set cut_decays True"
            param4 = "set ptb 30"
            param5 = "set ptj 20"
            param6 = "set ptl 20"
            param7 = "set etab 2.5"

            ruta_paramcard = "/disco4/personal_folders/Joaquin/Paramcards_13/" + "param_card_" + etiqueta + ".dat"

            lineas = [launch,nodos,shower,detector,done,param1, param2, param3, param4, param5, param6, param7,ruta_paramcard, ruta_pythia, ruta_delphes]

            script.write(lineas[0])

            for k in range(1,len(lineas)):    
                script.write('\n' + lineas[k])

            script.close()

            !./../../../Collider/MG5_aMC_v3_1_0/bin/mg5_aMC {nombre_script}
            !rm {nombre_script}
            
            source_folder = "/disco4/personal_folders/Joaquin/Data_300K_13_TeV/lq_ta/" + etiqueta + "/Events"
            target_folder = "/disco4/personal_folders/Joaquin/Data_300K_13_TeV/lq_ta/Events/" + etiqueta
            source_file_xs = "/disco4/personal_folders/Joaquin/Data_300K_13_TeV/lq_ta/" + etiqueta + "/crossx.html"
            target_file_xs = "/disco4/personal_folders/Joaquin/Data_300K_13_TeV/lq_ta/Cross_Sections/crossx_" + etiqueta + ".html"
            
            !rm -rf target_file_xs
            shutil.copy(source_file_xs, target_file_xs)
            !mkdir -p target_folder
            !rsync --modify-window=1 --update --recursive --copy-links {source_folder} {target_folder}                     

In [ ]:
print(seeds)

In [ ]:
for i in range(len(M)):
    for j in range(len(g_U)):
        etiqueta = "M" + M[i].replace('.','_') + "_gU" + g_U[j].replace('.','_')
        folder = "/disco4/personal_folders/Joaquin/Data_300K_13_TeV/lq_ta/" + etiqueta
        
        subprocesses_folder = folder + "/SubProcesses/"
        files = glob.iglob(os.path.join(subprocesses_folder, "**/*.ps"))

        for file_path in files:
            if os.path.isfile(file_path):
                ruta = os.path.dirname(os.path.abspath(file_path))
                for k in range(len(ruta)):
                    if (ruta[len(ruta) -1 - k] == "/"):
                        indice = len(ruta) -1 - k
                        subcarpeta = ruta[indice:]
                        break

                target_folder = f"/disco4/personal_folders/Joaquin/Data_300K_13_TeV/lq_ta/SubProcesses/{etiqueta}{subcarpeta}"
                !mkdir -p {target_folder}
                shutil.copy(file_path, target_folder)

                source_file_config_map = ruta +  "/config_subproc_map.inc"
                target_file_config_map = target_folder +  "/config_subproc_map.inc"
                shutil.copy(source_file_config_map, target_file_config_map)
        
        HTML_folder = folder + "/HTML/"
        files = glob.iglob(os.path.join(HTML_folder, "**/results.html"))

        for file_path in files:
            if os.path.isfile(file_path):
                ruta = os.path.dirname(os.path.abspath(file_path))
                for k in range(len(ruta)):
                    if (ruta[len(ruta) -1 - k] == "/"):
                        indice = len(ruta) -1 - k
                        subcarpeta = ruta[indice:]
                        break

                target_folder = f"/disco4/personal_folders/Joaquin/Data_300K_13_TeV/lq_ta/HTML/{etiqueta}{subcarpeta}"
                !mkdir -p {target_folder}
                shutil.copy(file_path, target_folder)            
        
        #shutil.rmtree(folder) #Borra la carpeta con los outputs de MadGraph

In [ ]:
xs_lq_ta = np.zeros((len(g_U), len(M)))
for j in range(len(M)):
    for i in range(len(g_U)):
        etiqueta = "M" + M[j].replace('.','_') + "_gU" + g_U[i].replace('.','_')
        
        ruta_tabla = "/disco4/personal_folders/Joaquin/Data_300K_13_TeV/lq_ta/Cross_Sections/crossx_" + etiqueta + ".html"
        tabla_ij = pd.read_html(ruta_tabla)
        columna_xs = tabla_ij[0]['Cross section (pb)']
        xs_lq_ta[i,j] = float(columna_xs[0].split(' ')[0])         
        
XS_Matriz_lq_ta = pd.DataFrame(xs_lq_ta)
XS_Matriz_lq_ta.columns = M
XS_Matriz_lq_ta.index = g_U

XS_Matriz_lq_ta.to_excel("/disco4/personal_folders/Joaquin/Data_300K_13_TeV/lq_ta/Cross_Sections/XS_Matriz.xlsx")